In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("TestApp").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/22 08:40:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/22 08:40:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
SRC = "/home/labuser/Downloads/Retail_Dataset/"

In [3]:
import glob

In [8]:
files = glob.glob(SRC+"/*.csv")

In [9]:
print(files)

['/home/labuser/Downloads/Retail_Dataset/customer.csv', '/home/labuser/Downloads/Retail_Dataset/region.csv', '/home/labuser/Downloads/Retail_Dataset/nation.csv', '/home/labuser/Downloads/Retail_Dataset/orders.csv', '/home/labuser/Downloads/Retail_Dataset/supplier.csv', '/home/labuser/Downloads/Retail_Dataset/partsupp.csv', '/home/labuser/Downloads/Retail_Dataset/part.csv', '/home/labuser/Downloads/Retail_Dataset/lineitem.csv']


In [16]:
Df = {"customer":"","region":"","nation":"","orders":"","supplier":"","partsupp":"","part":"","lineitem":""}

In [26]:
for item in zip(Df.keys(),files):
    Df[f'{item[0]}'] = spark.read.option("inferSchema",True).option("header",True).option("sep","\t").csv(item[1])

In [25]:
customerDf = spark.read.option("inferSchema",True).option("header",True).option("sep","\t").csv(files[0])

In [34]:
Df['lineitem'].show()

+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+-----------------+----------+--------------------+
|L_ORDERKEY|L_PARTKEY|L_SUPPKEY|L_LINENUMBER|L_QUANTITY|L_EXTENDEDPRICE|L_DISCOUNT|L_TAX|L_RETURNFLAG|L_LINESTATUS|L_SHIPDATE|L_COMMITDATE|L_RECEIPTDATE|   L_SHIPINSTRUCT|L_SHIPMODE|           L_COMMENT|
+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+-----------------+----------+--------------------+
|         1|     1552|       93|           1|      17.0|       24710.35|      0.04| 0.02|           N|           O|1996-03-13|  1996-02-12|   1996-03-22|DELIVER IN PERSON|     TRUCK|egular courts abo...|
|         1|      674|       75|           2|      36.0|       56688.12|      0.09| 0.06|           N|           O|1996-04-12|  1996-02-28|   1996-04-20| TAKE BACK RETURN|      MAIL|ly

## Joins 

In [35]:
test = Df['customer'].join(Df['orders'],Df['customer'].C_CUSTKEY == Df['orders'].O_CUSTKEY,how="inner")

In [37]:
test.select('C_CUSTKEY','O_ORDERKEY').show()

+---------+----------+
|C_CUSTKEY|O_ORDERKEY|
+---------+----------+
|      370|         1|
|      781|         2|
|     1234|         3|
|     1369|         4|
|      445|         5|
|      557|         6|
|      392|         7|
|     1301|        32|
|      670|        33|
|      611|        34|
|     1276|        35|
|     1153|        36|
|      862|        37|
|     1249|        38|
|      818|        39|
|      322|        64|
|      163|        65|
|     1292|        66|
|      568|        67|
|      286|        68|
+---------+----------+
only showing top 20 rows



In [39]:
type(Df['customer'])

pyspark.sql.dataframe.DataFrame

In [40]:
Df['orders'].dtypes

[('O_ORDERKEY', 'int'),
 ('O_CUSTKEY', 'int'),
 ('O_ORDERSTATUS', 'string'),
 ('O_TOTALPRICE', 'double'),
 ('O_ORDERDATE', 'date'),
 ('O_ORDERPRIORITY', 'string'),
 ('O_CLERK', 'string'),
 ('O_SHIPPRIORITY', 'int'),
 ('O_COMMENT', 'string')]

In [43]:
grpDf = Df['orders'].groupBy('O_CLERK').sum('O_TOTALPRICE')

In [44]:
grpDf.show()

+---------------+------------------+
|        O_CLERK| sum(O_TOTALPRICE)|
+---------------+------------------+
|Clerk#000000024|        2078084.41|
|Clerk#000000288|        1431028.03|
|Clerk#000000236|        2571896.97|
|Clerk#000000093|3388791.1500000004|
|Clerk#000000124|        1683034.58|
|Clerk#000000011|2032237.1699999995|
|Clerk#000000114|1596777.3599999999|
|Clerk#000000022|         1250757.7|
|Clerk#000000474|1669808.3399999999|
|Clerk#000000723| 923212.7899999999|
|Clerk#000000903|2984974.0700000008|
|Clerk#000000398|        1799587.31|
|Clerk#000000744|         2078169.2|
|Clerk#000000912|1728371.5500000005|
|Clerk#000000644|         2357482.5|
|Clerk#000000766|        1941942.28|
|Clerk#000000674|         941649.11|
|Clerk#000000821|2854880.0399999996|
|Clerk#000000424|1509396.7800000003|
|Clerk#000000087|        1396516.85|
+---------------+------------------+
only showing top 20 rows



In [48]:
Df['customer'].columns

['C_CUSTKEY',
 'C_NAME',
 'C_ADDRESS',
 'C_NATIONKEY',
 'C_PHONE',
 'C_ACCTBAL',
 'C_MKTSEGMENT',
 'C_COMMENT']

In [49]:
Df['orders'].columns

['O_ORDERKEY',
 'O_CUSTKEY',
 'O_ORDERSTATUS',
 'O_TOTALPRICE',
 'O_ORDERDATE',
 'O_ORDERPRIORITY',
 'O_CLERK',
 'O_SHIPPRIORITY',
 'O_COMMENT']

In [50]:
Df['lineitem'].columns

['L_ORDERKEY',
 'L_PARTKEY',
 'L_SUPPKEY',
 'L_LINENUMBER',
 'L_QUANTITY',
 'L_EXTENDEDPRICE',
 'L_DISCOUNT',
 'L_TAX',
 'L_RETURNFLAG',
 'L_LINESTATUS',
 'L_SHIPDATE',
 'L_COMMITDATE',
 'L_RECEIPTDATE',
 'L_SHIPINSTRUCT',
 'L_SHIPMODE',
 'L_COMMENT']

In [72]:
joinDf = Df['customer'].join(Df['orders'], on = Df['customer']["C_CUSTKEY"]==Df['orders']["O_CUSTKEY"],how="inner")\
                .join(Df['lineitem'], on = Df['lineitem']["L_ORDERKEY"]==Df['orders']["O_ORDERKEY"],how="inner")

In [73]:
joinDf.select('C_CUSTKEY','C_NAME','O_ORDERKEY','L_SHIPMODE','O_TOTALPRICE').show()

+---------+------------------+----------+----------+------------+
|C_CUSTKEY|            C_NAME|O_ORDERKEY|L_SHIPMODE|O_TOTALPRICE|
+---------+------------------+----------+----------+------------+
|      370|Customer#000000370|         1|      MAIL|   172799.49|
|      370|Customer#000000370|         1|       FOB|   172799.49|
|      370|Customer#000000370|         1|       AIR|   172799.49|
|      370|Customer#000000370|         1|   REG AIR|   172799.49|
|      370|Customer#000000370|         1|      MAIL|   172799.49|
|      370|Customer#000000370|         1|     TRUCK|   172799.49|
|      781|Customer#000000781|         2|      RAIL|    38426.09|
|     1234|Customer#000001234|         3|      RAIL|    205654.3|
|     1234|Customer#000001234|         3|       FOB|    205654.3|
|     1234|Customer#000001234|         3|     TRUCK|    205654.3|
|     1234|Customer#000001234|         3|      SHIP|    205654.3|
|     1234|Customer#000001234|         3|      RAIL|    205654.3|
|     1234

In [59]:
customerDf = Df['customer']
ordersDf = Df['orders']
lineitem = Df['lineitem']

In [67]:
result = customerDf.join(ordersDf,customerDf['C_CUSTKEY']==ordersDf['O_CUSTKEY'],how="inner")\
        .join(lineitem,ordersDf['O_ORDERKEY']==lineitem['L_ORDERKEY'],how="inner")

In [68]:
result.select('C_CUSTKEY','C_NAME','O_ORDERKEY','L_SHIPMODE','O_TOTALPRICE').show()

+---------+------------------+----------+----------+------------+
|C_CUSTKEY|            C_NAME|O_ORDERKEY|L_SHIPMODE|O_TOTALPRICE|
+---------+------------------+----------+----------+------------+
|      370|Customer#000000370|         1|      MAIL|   172799.49|
|      370|Customer#000000370|         1|       FOB|   172799.49|
|      370|Customer#000000370|         1|       AIR|   172799.49|
|      370|Customer#000000370|         1|   REG AIR|   172799.49|
|      370|Customer#000000370|         1|      MAIL|   172799.49|
|      370|Customer#000000370|         1|     TRUCK|   172799.49|
|      781|Customer#000000781|         2|      RAIL|    38426.09|
|     1234|Customer#000001234|         3|      RAIL|    205654.3|
|     1234|Customer#000001234|         3|       FOB|    205654.3|
|     1234|Customer#000001234|         3|     TRUCK|    205654.3|
|     1234|Customer#000001234|         3|      SHIP|    205654.3|
|     1234|Customer#000001234|         3|      RAIL|    205654.3|
|     1234

In [74]:
def concat_shell(columns):
    return columns.split("#")[0] + '_shell'

In [75]:
print(concat_shell('Customer#000000370'))

Customer_shell


In [76]:
from pyspark.sql.functions import *

In [77]:
function = udf(concat_shell,StringType())

In [79]:
testDf = result.withColumn("new_col",function(col("C_NAME")))

In [80]:
testDf.select('C_CUSTKEY','C_NAME','O_ORDERKEY','L_SHIPMODE','O_TOTALPRICE','new_col').show()

+---------+------------------+----------+----------+------------+--------------+
|C_CUSTKEY|            C_NAME|O_ORDERKEY|L_SHIPMODE|O_TOTALPRICE|       new_col|
+---------+------------------+----------+----------+------------+--------------+
|      370|Customer#000000370|         1|      MAIL|   172799.49|Customer_shell|
|      370|Customer#000000370|         1|       FOB|   172799.49|Customer_shell|
|      370|Customer#000000370|         1|       AIR|   172799.49|Customer_shell|
|      370|Customer#000000370|         1|   REG AIR|   172799.49|Customer_shell|
|      370|Customer#000000370|         1|      MAIL|   172799.49|Customer_shell|
|      370|Customer#000000370|         1|     TRUCK|   172799.49|Customer_shell|
|      781|Customer#000000781|         2|      RAIL|    38426.09|Customer_shell|
|     1234|Customer#000001234|         3|      RAIL|    205654.3|Customer_shell|
|     1234|Customer#000001234|         3|       FOB|    205654.3|Customer_shell|
|     1234|Customer#00000123

## How to Write the file in storage
    - Df.write.mode("append").csv("destination")

In [81]:
joinDf.write.mode("append").parquet("/home/labuser/Downloads/output/")

23/09/22 10:32:02 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


### Making partitions 

In [82]:
test = joinDf.repartition(8)

### It will create 8 new files

In [83]:
test.write.mode("append").parquet("/home/labuser/Downloads/output2/")

### Combining the partitioned file into one

In [84]:
test.coalesce(1).write.mode("append").parquet("/home/labuser/Downloads/output3/")